# Reinforcement Q-Learning from Scratch in Python with OpenAI Gym (обучение с подкреплением)
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [2]:
!pip install gym

In [2]:
import gym
import time #для управления временными задержками в коде
from IPython.display import clear_output #управления информацией в режиме реального времени

env = gym.make("Taxi-v3").env #создаем среду

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env.s = 328
env.reset() #сбрасывание среды к изначальному значению
env.render() #отображает текущее состояние среды в виде текстового представления
print(env.step(2))
time.sleep(10) #приостанавливает выполнение программы на 10 секунд
clear_output(wait=True)
env.render()

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


(383, -1, False, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})


# Решение без алгоритма RL
Выполнение случайных действий из каждого состояния

In [5]:
# Устанавливаем в ноль количество итераций, штрафы и вознаграждение
epochs = 0
penalties, reward = 0, 0

frames = [] # сохранение информации о каждом кадре среды для анимации

done = False

while not done:
    action = env.action_space.sample() #выбирает случайное действие
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Каждый отображенный текущий кадр помещаем в словарь/список для анимации
    frames.append({
        'frame': env.render(mode='ansi'),
        'episode': '0',
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Timesteps taken: {}".format(epochs)) #итерации
print("Penalties incurred: {}".format(penalties)) #штрафы

Timesteps taken: 26
Penalties incurred: 5


## Printing frames

In [6]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Episode: {frame['episode']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(.1)

In [8]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 0
Timestep: 26
State: 475
Action: 5
Reward: 20


# Обучение с подкреплением с использованием Q-Learning

In [10]:
import numpy as np

# инициализируем q- таблицу, заполняя ее нулями
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [11]:
%%time

import random
from IPython.display import clear_output

# устанавливаем гиперпараметры
alpha = 0.1 #коэффициент обучения
gamma = 0.6 #коэффициент дисконтирования
epsilon = 0.1 #вероятность выбора случайного действия

#построение метрики(показателей)
all_epochs = []
all_penalties = []

#обучение
for i in range(1, 100001):
    # перезагружаем среду, чтобы начать новый эпизод
    state = env.reset()

    # инициализируем переменные для текущего эпизода
    epochs, penalties, reward, = 0, 0, 0
    done = False

    # запускаем эпизод
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # пространство действий
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)

        # обновляем значение q для текущей пары состояние-действие
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        # выполняем действие и наблюдаем за следующим состоянием, наградой и статусом завершения
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        # обновить метрики, если наложен штраф
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    # печатаем номер эпизода каждые 100 эпизодов
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 16s, sys: 7.83 s, total: 1min 24s
Wall time: 1min 23s


# Evaluate agent's performance after Q-learning

In [12]:
total_epochs, total_penalties = 0, 0
episodes = 100
frames = []

for ep in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state]) #выбирает действие с наибольшим значением Q-таблицы для текущего состояния
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        # Каждый отображенный кадр помещаем в словарь для анимации
        frames.append({
            'frame': env.render(mode='ansi'),
            'episode': ep,
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.02
Average penalties per episode: 0.0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


# Visualization

In [13]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 99
Timestep: 1302
State: 0
Action: 5
Reward: 20
